# ISMN UPLOAD

This tool can be used to load a database dump from ISMN to a MetaCatalog instance. It is a tool-spec compliance 
specification and can be run like any other tool. **But it is only needed once**, to import the ISMN data into a 
Metacatalog instance.

**Read through the license information of ISMN and each of the networks before you use the data.**

## Setup and parameters

**Make sure that the following cell is tagged with `parameters`, otherwise papermill does not know where to inject the parameters as provided by the user of the tool.

In [ ]:
ismn_location = "/data/ismn"
#ismn_location = "../data/files/ismn"
data_pattern = "**/**/*.stm"

In [3]:
import os
import glob
from pathlib import Path
import warnings

from metacatalog_client.client import Client

# if you run this outside of a tool-spec container, create a local logger
#import logging as logger
#logger.basicConfig(level=logger.DEBUG)
from json2args.logger import logger

In [4]:
METACATALOG_URI = os.environ.get('METACATALOG_URI', 'http://localhost:8000/')
# check if the metacatalog instance can be found
client = Client(url=METACATALOG_URI)
logger.info(f"Connected to MetaCatalog v{client.host_version}")

[INFO]: Connected to MetaCatalog v0.3.9


## Load Metadata

In [5]:
all_files = glob.glob(ismn_location + '/' + data_pattern)
logger.info(f"The ISMN database dump contains {len(all_files)} STM files.")

[INFO]: The ISMN database dump contains 25349 STM files.


In [6]:
from tqdm import tqdm
import pandas as pd
import io

buffer = io.StringIO()
buffer.write("network,station,station_name,lat,lon,elevation,depth_from,depth_to,variable,device,filename\n")
def parse_file(path):
    with open(path, 'r') as f:
        raw_header = f.readline()
        header = [c.strip() for c in raw_header.split(' ') if c != '']

        # we overwrite the network and station information from the file, as these differ
        path_chunks = Path(path).name.split('_')
        
        # quote the network and station name
        header[0] = f'"{path_chunks[0]}"'
        header[1] = f'"{path_chunks[1]}"'
        header[2] = f'"{path_chunks[2]}"'
        static = ','.join(header[:8])
        device = ' '.join(header[8:]).replace(',', '_')
        line = f"{static},{path_chunks[3]},{device},\"{Path(path).name}\"\n"

        return line
        # return [*header[:8], ' '.join(header[8:]), Path(path).name]


for file_name in tqdm(all_files):
    header_line = parse_file(file_name)
    buffer.write(header_line)

buffer.seek(0)
raw_header = pd.read_csv(buffer, quotechar='"')

raw_header.insert(0, 'id', range(1, len(raw_header) + 1))
raw_header

100%|██████████| 25349/25349 [00:07<00:00, 3217.42it/s]


,id,network,station,station_name,lat,lon,elevation,depth_from,depth_to,variable,device,filename
0,1,AWDN,AWDN,NorthPlatte,41.05000,-100.46000,861.0,0.10,0.10,sm,ThetaProbe ML2X,AWDN_AWDN_NorthPlatte_sm_0.100000_0.100000_The...
1,2,AWDN,AWDN,NorthPlatte,41.05000,-100.46000,861.0,0.50,0.50,sm,ThetaProbe ML2X,AWDN_AWDN_NorthPlatte_sm_0.500000_0.500000_The...
2,3,AWDN,AWDN,NorthPlatte,41.05000,-100.46000,861.0,1.00,1.00,sm,ThetaProbe ML2X,AWDN_AWDN_NorthPlatte_sm_1.000000_1.000000_The...
3,4,AWDN,AWDN,NorthPlatte,41.05000,-100.46000,861.0,0.25,0.25,sm,ThetaProbe ML2X,AWDN_AWDN_NorthPlatte_sm_0.250000_0.250000_The...
4,5,AWDN,AWDN,Smithfield,40.35000,-99.40000,768.0,1.00,1.00,sm,ThetaProbe ML2X,AWDN_AWDN_Smithfield_sm_1.000000_1.000000_Thet...
...,...,...,...,...,...,...,...,...,...,...,...,...
25344,25345,IMA-CAN1,IMA-CAN1,station3,44.68241,8.62657,272.7,0.10,0.10,sm,5TM,IMA-CAN1_IMA-CAN1_station3_sm_0.100000_0.10000...
25345,25346,IMA-CAN1,IMA-CAN1,station10,44.68275,8.62636,278.5,0.10,0.10,sm,5TM,IMA-CAN1_IMA-CAN1_station10_sm_0.100000_0.1000...
25346,25347,IMA-CAN1,IMA-CAN1,station10,44.68275,8.62636,278.5,0.10,0.10,ts,5TM,IMA-CAN1_IMA-CAN1_station10_ts_0.100000_0.1000...
25347,25348,IMA-CAN1,IMA-CAN1,station11,44.68253,8.62671,272.6,0.10,0.10,sm,5TM,IMA-CAN1_IMA-CAN1_station11_sm_0.100000_0.1000...


In [7]:
print(raw_header.variable.unique())

# match these names to something we can use
# there is some info here: https://ismn.earth/media/filer_public/1f/4f/1f4f1b03-550b-4b63-b680-fc9695d6feec/data_template_description_28082023.pdf
# sm = soil moisture
# ts = soil temperature
# ta = air temperature
# p = precipitation
# sd = snow-depth
# sweq = snow water equivalent
lookup = {
    'sm': 12,
    'ts': 2,
    'ta': 1,
    'p': 8,
    'su': 15
}

variables = {short: client.variable(id=id) for short, id in lookup.items()}
variables

['sm' 'ts' 'ta' 'p' 'sd' 'tsf' 'sweq' 'su']


{'sm': {'name': 'volumetric water content',
  'symbol': 'theta',
  'column_names': ['volumetric_water_content'],
  'id': 12,
  'unit': {'name': 'cm3/cm3', 'symbol': 'cm3/cm3', 'si': '1'},
  'keyword': {'id': 5727,
   'uuid': 'bbe2ea34-8842-4a9f-9b0b-95dd3c71857f',
   'parent_id': 158,
   'value': 'SOIL MOISTURE/WATER CONTENT',
   'full_path': 'EARTH SCIENCE > LAND SURFACE > SOILS > SOIL MOISTURE/WATER CONTENT',
   'thesaurus': {'name': 'GCMD',
    'title': 'NASA/GCMD Earth Science Keywords',
    'uuid': '2e54668d-8fae-429f-a511-efe529420b12',
    'description': 'NASA Global Clime change Master Dictionary Science Keywords',
    'organisation': 'NASA',
    'url': 'https://gcmdservices.gsfc.nasa.gov/kms/concepts/concept_scheme/sciencekeywords/?format=xml'}}},
 'ts': {'name': 'soil temperature',
  'symbol': 'Ts',
  'column_names': ['soil_temperature'],
  'id': 2,
  'unit': {'name': 'degree Celsius', 'symbol': 'C', 'si': 'K'},
  'keyword': {'id': 5736,
   'uuid': '0546b91a-294d-45d9-8b45-76

## Load static properties

In [8]:
all_meta = glob.glob(ismn_location + '/' + data_pattern.replace('.stm', '.csv'))
logger.info(f"Found {len(all_meta)} files with static attributes. Exciting.")

[INFO]: Found 3210 files with static attributes. Exciting.


In [9]:
def get_static_attributes(path):
    columns = ['quantity_name', 'unit', 'depth_from', 'depth_to','value','description','quantity_source_name', 'quantity_source_description', 'quantity_source_provider', 'quantity_source_version', 'quantity_source_resolution', 'quantity_source_timerange', 'quantity_source_url']

    # get the network and station name from the file name
    network, station, station_name, *_ = Path(path).name.split('_')

    # check if the file has a valid header
    with open(path, 'r') as f:
        if not f.readline().startswith(columns[0]):
            attrs = pd.read_csv(path, sep=';', header=None, names=columns)
        else:
            attrs = pd.read_csv(path, sep=';')

    # add the network and station identifiers to the attributes
    attrs.insert(0, 'network', network)
    attrs.insert(1, 'station', station)
    attrs.insert(2, 'station_name', station_name)

    # rename depth_from and depth_to columns
    attrs.rename(columns={'depth_from[m]': 'depth_from', 'depth_to[m]': 'depth_to'}, inplace=True)
    
    # return
    #return attrs.dropna(axis='columns', how='all')
    return attrs

statics = get_static_attributes(all_meta[0]) 

In [10]:
from tqdm import tqdm

n_errors = 0
error_msgs = []
static_blacklist = []

raw_statics = None
for meta_file in tqdm(all_meta):
    # load the static properties
    statics = get_static_attributes(meta_file)
    
    # run a few checks
    # check that the depth_from column is numric
    if not statics.depth_from.dtype == 'float64':
        static_blacklist.append(meta_file)
        error_msgs.append(f"{meta_file}: depth_from is not numeric")
        n_errors += 1
        continue
    # check that the depth_to column is numric
    if not statics.depth_to.dtype == 'float64':
        static_blacklist.append(meta_file)
        error_msgs.append(f"{meta_file}: depth_to is not numeric")
        n_errors += 1
        continue
    
    # pd only
    if raw_statics is None:
        raw_statics = statics.copy()
    else:
        raw_statics = pd.concat([raw_statics, statics], ignore_index=True, axis='rows')


print(f"Ran into {n_errors} errors.")
raw_statics

100%|██████████| 3210/3210 [00:08<00:00, 376.27it/s]

Ran into 55 errors.


,network,station,station_name,quantity_name,unit,depth_from,depth_to,value,description,quantity_source_name,quantity_source_description,quantity_source_provider,quantity_source_version,quantity_source_resolution,quantity_source_timerange,quantity_source_url,Unnamed: 13
0,AWDN,AWDN,NorthPlatte,saturation,m^3*m^-3,0.0,0.3,0.46,NaN,HWSD,Harmonized World Soil Database v1.1 by IIASA,IIASA,v1.1,"30""",NaN,http://webarchive.iiasa.ac.at/Research/LUC/Ext...,NaN
1,AWDN,AWDN,NorthPlatte,clay fraction,% weight,0.0,0.3,23.00,NaN,HWSD,Harmonized World Soil Database v1.1 by IIASA,IIASA,v1.1,"30""",NaN,http://webarchive.iiasa.ac.at/Research/LUC/Ext...,NaN
2,AWDN,AWDN,NorthPlatte,organic carbon,% weight,0.0,0.3,1.27,NaN,HWSD,Harmonized World Soil Database v1.1 by IIASA,IIASA,v1.1,"30""",NaN,http://webarchive.iiasa.ac.at/Research/LUC/Ext...,NaN
3,AWDN,AWDN,NorthPlatte,sand fraction,% weight,0.0,0.3,36.00,NaN,HWSD,Harmonized World Soil Database v1.1 by IIASA,IIASA,v1.1,"30""",NaN,http://webarchive.iiasa.ac.at/Research/LUC/Ext...,NaN
4,AWDN,AWDN,NorthPlatte,silt fraction,% weight,0.0,0.3,41.00,NaN,HWSD,Harmonized World Soil Database v1.1 by IIASA,IIASA,v1.1,"30""",NaN,http://webarchive.iiasa.ac.at/Research/LUC/Ext...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56339,IMA-CAN1,IMA-CAN1,station11,soil classification,NaN,NaN,NaN,NaN,Dystric Cambisol: UTS according to WRB,insitu,in situ,NaN,NaN,NaN,NaN,NaN,NaN
56340,IMA-CAN1,IMA-CAN1,station11,land cover classification,NaN,NaN,NaN,10,"Cropland, rainfed (Level 1, CCI-LC)",insitu,in situ,NaN,NaN,NaN,NaN,NaN,NaN
56341,IMA-CAN1,IMA-CAN1,station11,climate classification,NaN,NaN,NaN,Csa,KG according to version of March 2017 Rubel e...,insitu,in situ,NaN,NaN,NaN,NaN,NaN,NaN
56342,IMA-CAN1,IMA-CAN1,station11,climate classification,NaN,NaN,NaN,Cfa,Temperate - Without dry season - Hot Summer,koeppen_geiger_2007,Koeppen-Geiger Climate Classification,NaN,Peel2007,0.1°,NaN,http://www.hydrol-earth-syst-sci.net/11/1633/2...,NaN


In [10]:
for msg in error_msgs:
    logger.error(msg)

[ERROR]: /data/ismn/RUSWET-AGRO/KrasnojarskiKrai#2/RUSWET-AGRO_RUSWET-AGRO_KrasnojarskiKrai#2_static_variables.csv: depth_to is not numeric
[ERROR]: /data/ismn/RUSWET-AGRO/Gurjvskaya/RUSWET-AGRO_RUSWET-AGRO_Gurjvskaya_static_variables.csv: depth_to is not numeric
[ERROR]: /data/ismn/RUSWET-AGRO/KrasnojarskiKrai#1/RUSWET-AGRO_RUSWET-AGRO_KrasnojarskiKrai#1_static_variables.csv: depth_to is not numeric
[ERROR]: /data/ismn/TERENO/Merzenhausen/TERENO_TERENO_Merzenhausen_static_variables.csv: depth_from is not numeric
[ERROR]: /data/ismn/PTSMN/Site-2/PTSMN_PTSMN_Site-2_static_variables.csv: depth_from is not numeric
[ERROR]: /data/ismn/COSMOS-UK/Writtle/COSMOS-UK_COSMOS-UK_Writtle_static_variables.csv: depth_to is not numeric
[ERROR]: /data/ismn/COSMOS-UK/Cochno/COSMOS-UK_COSMOS-UK_Cochno_static_variables.csv: depth_to is not numeric
[ERROR]: /data/ismn/RISMA/ON2/RISMA_RISMA_ON2_static_variables.csv: depth_from is not numeric
[ERROR]: /data/ismn/RUSWET-AGRO/Gurjvskaya/RUSWET-AGRO_RUSWET-AGR

## Test to parse the Readme


The Readme generally looks like this with a few formatting issues
Intendation **cannot** be used as sometimes the references break over.

```
	STEMS
		Abstract: Soil Moisture Network installed in rainfed vineyard plots, with different inter-rows soil management. The plots are also monitored for runoff and soil erosion. Weather data available from a station near the plots.
		Continent: Europe
		Country: Italy
		Stations: 4
		Status: running
		Data Range: from 2015-12-04 
		Type: campaign
		Url: https://sustag.to.cnr.it/index.php/cannona-db
		Reference: Darouich, H., Ramos, T.B., Pereira, L.S., Rabino, D., Bagagiolo, G., Capello, G., Simionesei, L., Cavallo, E., Biddoccu, M. Water Use and Soil Water Balance of Mediterranean Vineyards under Rainfed and Drip Irrigation Management: Evapotranspiration Partition and Soil Management Modelling for Resource Conservation. Water 2022, 14, 554. https://doi.org/10.3390/w14040554;

Capello G, Biddoccu M, Ferraris S, Cavallo E, 2019. Effects of tractor passes on hydrological and soil erosion processes in tilled and grassed vineyards. Water 2019, 11(10), 2118, https://doi.org/10.3390/w11102118;
		Variables: air temperature, precipitation, soil moisture, soil temperature, 
		Soil Moisture Depths: 0.10 - 0.10 m, 0.20 - 0.20 m, 0.30 - 0.30 m, 0.40 - 0.40 m, 0.50 - 0.50 m
		Soil Moisture Sensors: 5TM, EC5, 

```

Additionally, there are a lot of places, where other additional comments are placed out of formatting. They are logged out below, but will not be read into the database.

In [11]:
# define the fields
fields = ['Abstract', 'Continent', 'Country', 'Stations', 'Status', 'Data Range', 'Type', 'Url', 'Reference', 'Variables', 'Soil Moisture Depths', 'Soil Moisture Sensors']

with open(ismn_location + '/Readme.txt') as f:
    readme = f.read()

logger.info("Trying to parse the Readme.txt for Station descriptions. This is going to be a mess.")
header_idx = 0
for line in readme.splitlines():
    # skip over everything until we reach the Network Information section
    if not line.strip() == 'Network Information':
        header_idx += 1
    else:
        break

current_network = None
current_data = {}
in_reference = False
networks = {}
for i, line in enumerate(readme.splitlines()):
    # if we are in the header or the line is empty, continue
    if i < header_idx or line.strip() == "":
        continue

    # if we are not in a network, the next line will be a network name
    if current_network is None:
        current_network = line.strip()
        continue

    # handle the reference
    if in_reference:
        # make sure we did not yet reach the Variable
        if not line.strip().startswith('Variables:'):
            current_data['Reference'] += f"\n{line.strip()}"
            continue
        else:
            in_reference = False
    
    # we are in a network and need to append the data
    chunks = line.split(':')
    ident = chunks[0].strip()
    if ident in fields:
        current_data[ident] = ':'.join(chunks[1:]).strip()
        
        # handle the reference field
        if ident == 'Reference':
            in_reference = True

        # handle the end of a network section
        if ident == fields[-1]:
            networks[current_network.lower()] = current_data
            current_data = {}
            in_reference = False
            current_network = None

    else:
        logger.info(f"Readme.txt L.{i + 1}: Unkwnown field: {ident} ")

logger.info(f"Parsed {len(networks)} network descriptions")

[INFO]: Trying to parse the Readme.txt for Station descriptions. This is going to be a mess.
[INFO]: Readme.txt L.63: Unkwnown field: AACES 
[INFO]: Readme.txt L.164: Unkwnown field: This project contributes to understanding of the structure, function, and dynamics of boreal forest ecosystems and the broader boreal landscape, including the human communities. It assembles and integrates valuable long-term data sets on climate, hydrology, biology, ecology, and biogeochemical and geomorphic processes, as incorporates emerging data types, including molecular and social science data and digital images. The project has broad societal value through its contributions to knowledge that can inform management of boreal forest ecosystems and sustainability of subsistence communities. Its broader values also include extensive research-based training and educational program development. Its strong public outreach program includes collaborations between artists and scientists and strong linkages with

In [12]:
# print out one of the networks
networks[list(networks.keys())[0]]

{'Abstract': '',
 'Continent': 'Oceania',
 'Country': 'Australia',
 'Stations': '49',
 'Status': 'inactive',
 'Data Range': 'from 2010-01-18  to 2010-09-26',
 'Type': 'project',
 'Url': 'http://www.moisturemap.monash.edu.au/',
 'Reference': 'Peischl, S., Walker, J. P., Rüdiger, C., Ye, N., Kerr, Y. H., Kim, E., Bandara, R., and Allahmoradi, M.: The AACES field experiments: SMOS calibration and validation across the Murrumbidgee River catchment, Hydrology and Earth System Sciences, Discuss., 9, 2763-2795, https://doi.org/10.5194/hess-16-1697-2012, 2012;',
 'Variables': 'precipitation, soil temperature, soil moisture,',
 'Soil Moisture Depths': '0.00 - 0.05 m, 0.00 - 0.06 m, 0.25 - 0.25 m',
 'Soil Moisture Sensors': 'ThetaProbe ML2X,'}

## Read the ISMN dump

The next section defines the functions needed to read in the ISMN metadata.

In [13]:
def get_static(row: pd.Series, raw_statics: pd.DataFrame) -> pd.DataFrame:
    df = raw_statics.where(raw_statics.network == row.network).where(raw_statics.station == row.station).where(raw_statics.station_name == row.station_name)
    df = df.where((df.depth_from <= row.depth_from) & (df.depth_to >= row.depth_to) | df.depth_from.isnull())

    return df.dropna(axis='rows', how='all')

def get_details(static_attributes: pd.DataFrame) -> list[dict]:
    details = []
    for i, d in static_attributes.iterrows():
        # get the key
        k = d.quantity_source_name if 'classification' in d.quantity_name else d.quantity_name
        
        # count if this key is already in use - that can actually happen
        other_keys = [d['key'] for d in details if d['key'] == k]
        if len(other_keys) > 0:
            k = f"{k} [duplicate #{len(other_keys)}]"
        
        # get the value and the stem and append
        v = d.description if 'classification' in d.quantity_name else float(d.value)
        details.append(dict(key=k, raw_value=v))
    return details

def get_datasource(row: pd.Series, variable: dict) -> dict:
    # build the path
    p = Path(ismn_location) / row.station / row.station_name / row.filename
    
    # build the args
    args = dict(
        skiprows=1,
        sep='\s+',
        header=None,
        parse_dates={'tstamp': [0, 1]},
        names=['date', 'time', variable['name'].replace(' ', '_'), 'g', 'q']
    )
    # read the data in
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        _d = pd.read_csv(p, **args)
    
    # check if the file is not empty
    if _d.empty:
        raise ValueError("The file is empty.")
    elif len(_d) == 1:
        raise ValueError("The file only contains one row.")
    
    # build the temporal scale
    t_scale = dict(
        dimension_names=['tstamp'],
        observation_start=_d.tstamp.min().to_pydatetime(),
        observation_end=_d.tstamp.max().to_pydatetime(),
        resolution=pd.to_timedelta(_d.tstamp.diff().mode().values[0]),
        support=1.0
    )

    # build the datasource
    datasource = dict(
        path=str(p),
        type=3,
        variable_names=[variable['name'].replace(' ', '_')],
        temporal_scale=t_scale,
        args=args
    )

    return datasource

def get_metadata(row: pd.Series, variable: dict, raw_statics: pd.DataFrame) -> dict:
    # get the static attributes
    statics = get_static(row, raw_statics=raw_statics)

    # get the details
    details = [
        dict(key="depth_from", raw_value=row.depth_from),
        dict(key="depth_to", raw_value=row.depth_to),
        *get_details(statics)
    ]

    # handle the abstract.
    default_abstract = f"This is an auto-generated abstract. No information could be found in ISMN dump.\nStatic attributes:\n\n{statics.to_markdown()}"
    if row.station.lower() in networks:
        nw = networks[row.station.lower()]
        abstract = nw['Abstract']
        citation = nw['Reference']   
    else:
        abstract = ""
        citation = ""

    payload = dict(
        title=f"{row.station} {row.station_name} {variable['name']} ({row.depth_from}-{row.depth_to}) [{row.id}]",
        abstract=abstract if abstract != "" else default_abstract,
        details=details,
        variable=variable['id'],
        #location=f"SRID=4326;POINT({row.lon} {row.lat})"
        location=f"POINT({row.lon} {row.lat})",
        citation=citation if citation != "" else None
    )

    return payload

<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_9/1790871432.py:30: SyntaxWarning: invalid escape sequence '\s'
  sep='\s+',


In [14]:
# just try any of the entries
row = raw_header.iloc[410,:]

# first get the variable
variable = variables[row.variable]
meta = get_metadata(row, variable, raw_statics=raw_statics)
datasource=get_datasource(row, variable)
meta['datasource'] = datasource
print(datasource)
meta['citation']
meta['details']


{'path': '/data/ismn/RUSWET-AGRO/Gbao/RUSWET-AGRO_RUSWET-AGRO_Gbao_sm_0.000000_1.000000_n.s._19500101_20241119.stm', 'type': 3, 'variable_names': ['volumetric_water_content'], 'temporal_scale': {'dimension_names': ['tstamp'], 'observation_start': datetime.datetime(1992, 5, 18, 12, 0), 'observation_end': datetime.datetime(1994, 4, 28, 12, 0), 'resolution': Timedelta('10 days 00:00:00'), 'support': 1.0}, 'args': {'skiprows': 1, 'sep': '\\s+', 'header': None, 'parse_dates': {'tstamp': [0, 1]}, 'names': ['date', 'time', 'volumetric_water_content', 'g', 'q']}}


[{'key': 'depth_from', 'raw_value': 0.0},
 {'key': 'depth_to', 'raw_value': 1.0},
 {'key': 'CCI_landcover_2000', 'raw_value': 'Grassland'},
 {'key': 'CCI_landcover_2005', 'raw_value': 'Grassland'},
 {'key': 'CCI_landcover_2010', 'raw_value': 'Grassland'},
 {'key': 'koeppen_geiger_2007', 'raw_value': 'Cold - Dry Summer - Hot Summer'},
 {'key': 'koeppen_geiger_2017', 'raw_value': 'Arid - Desert - Cold'}]

In [15]:
from json2args.logger import logger

# set an author
ismn = dict(organisation_name="International Soil Moisture Network", organisation_abbrev="ISMN", is_organisation=True)

# handle license and variable
client.set_static(license=9)
client.set_static(author=ismn)

for _, row in tqdm(raw_header.iterrows(), total=len(raw_header)):
    # first get the variable
    variable = variables.get(row.variable)
    if variable is None:
        logger.info(f"[{row.filename}] Skipping as the variable {row.variable} is currently not supported.")
        continue

    # build the title
    title=f"{row.station} {row.station_name} {variable['name']} ({row.depth_from}-{row.depth_to}) [{row.id}]"
    metas = client.entries(title=title)
    if len(metas) > 0:
        meta = metas[0]
        entry_id = meta['id']
    else:
        # parse the metadata
        meta = get_metadata(row, variable, raw_statics=raw_statics)

    if meta.get('datasource') is None:
        # parse the datasource
        try:
            datasource = get_datasource(row, variable)
        except ValueError as e:
            if 'is empty' in str(e) or 'one row' in str(e):
                logger.info(f"[{row.filename}]: Skipping: {str(e)}")
                continue

        # add the meta
        try:
            entry = client.create_entry(
                title=meta['title'],
                abstract=meta['abstract'],
                citation=meta['citation'],
                variable=meta['variable'],
                details=meta['details'],
                location=meta.get('location'),
                datasource=datasource,
                duplicate_authors=False
            )

        except Exception as e:
            logger.error(f"Errored uploading {row.filename} to database: {str(e)}")
            continue

        #TODO: here we need the upgrade script (check extent)

  3%|▎         | 688/25349 [03:00<1:47:37,  3.82it/s]
/tmp/ipykernel_9/1790871432.py:30: SyntaxWarning: invalid escape sequence '\s'
  sep='\s+',


KeyboardInterrupt: 